In [2]:
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pickle
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
from pathlib import Path
from copy import deepcopy
from itertools import count

In [5]:
import easyrl
from easyrl.utils import *

In [8]:
# import sys
from radar_maps.env.radar_map_double_integrator import RadarMap_DoubleIntegrator

In [10]:
# from numba import jit

In [9]:
data_path = "double_integrator/data_multimodal/"
data_num = 4
num_mode = 3

In [10]:
class Trajectory:
    def __init__(self, obs, actions):
        self.obs = obs # Observations
        self.actions = actions # Actions

        
class TrajDataset(Dataset):
    def __init__(self, trajs):
        states = []
        actions = []
        for traj in trajs:
            states.extend(traj.obs)
            actions.extend(traj.actions)

        self.states = np.array(states)
        self.actions = np.array(actions)

    def __len__(self):
        return self.actions.shape[0]

    def __getitem__(self, idx):
        sample = dict()
        sample['state'] = self.states[idx]['state']
        sample['img'] = self.states[idx]['img']
        sample['action'] = self.actions[idx]
        return sample
    

In [11]:
# @jit
def get_path_chunk(path, num_points=3):
    len_path = len(path)
    if len_path >= num_points:
        return list(np.hstack(path[:num_points]))
    
    path_patched = list(np.hstack(path))
    for _ in range(num_points-len_path):
        path_patched.extend(list(path[-1]))
    # print("Patched path: ", path_patched)
    return path_patched

# @jit
def get_radar_heat_map(state, radar_locs, img_size, radar_detection_range, grid_size):
    radars_encoding = np.zeros((img_size, img_size))
    theta = np.arctan2(state[3], state[1])
    loc_to_glob = np.array([[np.cos(theta), -np.sin(theta), state[0]],
                            [np.sin(theta), np.cos(theta), state[2]],
                            [0., 0., 1.]])
    glob_to_loc = np.linalg.inv(loc_to_glob)
    # print(glob_to_loc)
    for radar_loc in radar_locs:
        if abs(state[0] - radar_loc[0]) < radar_detection_range or abs(state[2] - radar_loc[1]) < radar_detection_range:
            glob_loc_hom = np.array([radar_loc[0], radar_loc[1], 1])
            local_loc_hom = np.dot(glob_to_loc, glob_loc_hom)
            radars_loc_coord = local_loc_hom[:2]

            y_grid = np.rint((radars_loc_coord[1]) / grid_size) 
            x_grid = np.rint((radars_loc_coord[0]) / grid_size) 

            for i in range(-int(img_size/2), int(img_size/2)):
                for j in range(-int(img_size/2), int(img_size/2)):
                    radars_encoding[int(i + img_size/2), int(j + img_size/2)] += np.exp((-(x_grid - i)**2 - (y_grid - j)**2))*1e3

    radars_encoding = radars_encoding.T
    if np.max(radars_encoding) > 0:
        formatted = (radars_encoding * 255.0 / np.max(radars_encoding)).astype('float32')
    else:
        formatted = radars_encoding.astype('float32')

    formatted = formatted[np.newaxis, :, :]

    return formatted

# @jit
def generate_training_data(traj, ctr, path_mm, radars, detection_range, grid_size, v_lim, u_lim):
    observations = []
    actions = []
    # print("Len:", traj.shape)
    for i in range(traj.shape[0]-1):
        x_cur = traj[i]

        heat_map_img = get_radar_heat_map(x_cur, radars, 2*int(detection_range/grid_size), detection_range, grid_size)
        # print(heat_map_img.shape)
        x_cur_normalized = np.array([x_cur[0]/1200.0, x_cur[1]/v_lim, x_cur[2]/1200.0, x_cur[3]/v_lim])

        observation = {"state": x_cur_normalized, "img": heat_map_img}
        observations.append(observation)
        # print("Iter: ", i)
        # print("Observation: ", observations)
        if i < traj.shape[0] - 1:
            action_prediction = []
            for m in range(num_mode):
                action_prediction.extend(ctr[i, 2*m:2*(m+1)]/u_lim)
                path_tmp = path_mm[num_mode*i + m]
                path_tmp = [x / 1200.0 for x in path_tmp]
                # print(path_tmp)
                action_prediction.extend(get_path_chunk(path_tmp))
            actions.append(action_prediction)
    # print("Observation shape: ", np.array(observations).shape)
    return np.array(observations), np.array(actions)

def process_data(detection_range, grid_size, v_lim, u_lim):
    bc_data = []
    for i in range(data_num):
        print("Processing data: ", i)
        traj = np.load(data_path + f'state_traj_{i}.npy')
        control = np.load(data_path + f'control_traj_{i}.npy')
        radar_config = np.load(data_path + f'radar_config_{i}.npy')

        with open(data_path+ f'nominal_path_multimodal_{i}.pkl', 'rb') as handle:
            path_mm = pickle.load(handle)

        obs, acts = generate_training_data(traj, control, path_mm, radar_config, detection_range, grid_size, v_lim, u_lim)
        bc_traj = Trajectory(obs, acts)
        bc_data.append(bc_traj)
    return np.array(bc_data)

In [12]:
# import gym
from typing import Dict

In [13]:
from dataclasses import dataclass

In [14]:
def set_random_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)

# set random seed
seed = 0
set_random_seed(seed=seed)

In [15]:
import os
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

In [16]:
from easyrl import configs

In [17]:
def set_configs(exp_name='bc'):
    configs.set_config('ppo')
    configs.cfg.alg.seed = seed
    configs.cfg.alg.num_envs = 1
    configs.cfg.alg.episode_steps = 150
    configs.cfg.alg.max_steps = 600000
    configs.cfg.alg.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # configs.cfg.alg.device = 'cpu'
    configs.cfg.alg.env_name = 'RadarMap-DoubleIntegrator-v0'
    configs.cfg.alg.save_dir = Path.cwd().absolute().joinpath('data').as_posix()
    configs.cfg.alg.save_dir += f'/{exp_name}'
    setattr(configs.cfg.alg, 'diff_cfg', dict(save_dir=configs.cfg.alg.save_dir))

    print(f'====================================')
    print(f'      Device:{configs.cfg.alg.device}')
    print(f'====================================')

In [18]:
set_configs()

[INFO][2024-03-04 13:50:58]: Alogrithm type:<class 'easyrl.configs.ppo_config.PPOConfig'>


      Device:cpu


In [19]:
import torch.optim as optim
from tqdm.notebook import tqdm

In [20]:
import itertools

In [21]:
def safe_to_tensor(array, **kwargs):
    if isinstance(array, torch.Tensor):
        return array

    if not array.flags.writeable:
        array = array.copy()

    return torch.as_tensor(array, **kwargs)


In [22]:
def train_bc_agent(agent, trajs, max_epochs=5000, batch_size=32, lr=0.001, disable_tqdm=True, entropy_weight=1e-3):
    dataset = TrajDataset(trajs)
    dataloader = DataLoader(dataset, 
                            batch_size=batch_size, 
                            shuffle=True, 
                            drop_last=True)
    # print("Dataset shape ", dataset.actions.shape)
    optimizer = optim.Adam(agent.parameters(),
                           lr=lr)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    pbar = tqdm(range(max_epochs), desc='Epoch', disable=disable_tqdm)
    logs = dict(loss=[], epoch=[])
    
    for iter in pbar:
        avg_loss = []
        for batch_idx, sample in enumerate(dataloader):
            states = sample['state'].float().to(configs.cfg.alg.device)
            imgs = sample['img'].float().to(configs.cfg.alg.device)
            expert_actions = sample['action'].float().to(configs.cfg.alg.device)
            optimizer.zero_grad()
            # print(expert_actions.shape)
            _, act_dist = agent.forward({'state': states, 'img': imgs})
            
            log_prob = act_dist.log_prob(expert_actions)
            log_prob = log_prob.mean()

            entropy = act_dist.entropy()
            entropy = entropy.mean() if entropy is not None else None
            entropy_loss = -entropy_weight * (entropy if entropy is not None else torch.zeros(1))
            # print(actions)
            '''
            Handle trajectory assignment.
            '''
            indx1 = range(8)
            indx2 = range(8, 16)
            indx3 = range(16, 24)
            indx = [indx1, indx2, indx3]
            perm_indx = itertools.permutations([0, 1, 2], 3) 
            # print("Expert actions: ", expert_actions[0])
            for ind in list(perm_indx):
                indices = []
                indices.extend(indx[ind[0]])
                indices.extend(indx[ind[1]])
                indices.extend(indx[ind[2]])
                log_prob_permutated = act_dist.log_prob(expert_actions[:,  torch.tensor(indices)])
                log_prob_permutated = log_prob_permutated.mean()
                # print(loss_permutated.grad_fn)
                if log_prob_permutated > log_prob:
                    log_prob = torch.clone(log_prob_permutated)
            ####
            neglogp = -log_prob
            loss = neglogp + entropy_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.set_postfix({'loss': loss.item()})
            avg_loss.append(loss.item())
        
            # print("Optimizer param: ", optimizer.param_groups)
        scheduler.step()
        print(f'Epoch {iter} Loss: ', np.mean(avg_loss))
        logs['loss'].append(np.mean(avg_loss))
        logs['epoch'].append(iter)
    return agent, logs, len(dataset)

In [23]:
size_of_map = 1000
detection_range = 300
grid_size = 5
env = RadarMap_DoubleIntegrator(size_of_map, [size_of_map, size_of_map], detection_range, grid_size, dist_between_radars=size_of_map/5.0, num_radars=10)

AttributeError: module 'scipy.stats.qmc' has no attribute 'PoissonDisk'

In [25]:
from actor_utils import ActorNet

In [26]:
agent = ActorNet(env.action_space, env.observation_space, [64, 64])

In [27]:
print(agent)

ActorNet(
  (feature_extractor): FeatureExtractor(
    (extractors): ModuleDict(
      (img): NatureCNN(
        (cnn): Sequential(
          (0): Conv2d(1, 32, kernel_size=(8, 8), stride=(4, 4))
          (1): ReLU()
          (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
          (3): ReLU()
          (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
          (5): ReLU()
          (6): Flatten(start_dim=1, end_dim=-1)
        )
        (linear): Sequential(
          (0): Linear(in_features=7744, out_features=256, bias=True)
          (1): ReLU()
        )
      )
      (state): Flatten(start_dim=1, end_dim=-1)
    )
  )
  (policy_net): Sequential(
    (0): Linear(in_features=260, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
  )
  (action_net): Linear(in_features=64, out_features=24, bias=True)
)


In [34]:
bc_data = process_data(detection_range, grid_size, 20.0, 2.0)

Processing data:  0
Processing data:  1
Processing data:  2
Processing data:  3


In [35]:
from functools import partial

In [36]:
def init_weights(module: nn.Module, gain: float = 1) -> None:
    """
    Orthogonal initialization (used in PPO and A2C)
    """
    if isinstance(module, (nn.Linear, nn.Conv2d)):
        nn.init.orthogonal_(module.weight, gain=gain)
        if module.bias is not None:
            module.bias.data.fill_(0.0)

In [37]:
module_gains = {
    agent.feature_extractor: np.sqrt(2),
    agent.policy_net: np.sqrt(2),
    agent.action_net: 0.01,
}

for module, gain in module_gains.items():
    module.apply(partial(init_weights, gain=gain))

In [38]:
if torch.cuda.is_available():
    agent.cuda()

In [39]:
print(bc_data.shape)

(4,)


In [40]:
agent, logs, _= train_bc_agent(agent, bc_data, max_epochs=20)

Epoch 0 Loss:  23.26210525708321
Epoch 1 Loss:  21.476908903855545
Epoch 2 Loss:  20.281897520407654
Epoch 3 Loss:  19.44217892182179
Epoch 4 Loss:  18.713183916532078
Epoch 5 Loss:  18.079755098391804
Epoch 6 Loss:  17.52592932872283
Epoch 7 Loss:  17.029985330043694
Epoch 8 Loss:  16.594215295253655
Epoch 9 Loss:  16.202442560440456
Epoch 10 Loss:  15.845841285509941
Epoch 11 Loss:  15.528942841749926
Epoch 12 Loss:  15.248218634189703
Epoch 13 Loss:  14.99305177346254
Epoch 14 Loss:  14.755873900193434
Epoch 15 Loss:  14.53539202763484
Epoch 16 Loss:  14.33320783957457
Epoch 17 Loss:  14.145398017687675
Epoch 18 Loss:  13.97642402159862
Epoch 19 Loss:  13.824726691612831


In [41]:
agent.eval()

ActorNet(
  (feature_extractor): FeatureExtractor(
    (extractors): ModuleDict(
      (img): NatureCNN(
        (cnn): Sequential(
          (0): Conv2d(1, 32, kernel_size=(8, 8), stride=(4, 4))
          (1): ReLU()
          (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
          (3): ReLU()
          (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
          (5): ReLU()
          (6): Flatten(start_dim=1, end_dim=-1)
        )
        (linear): Sequential(
          (0): Linear(in_features=7744, out_features=256, bias=True)
          (1): ReLU()
        )
      )
      (state): Flatten(start_dim=1, end_dim=-1)
    )
  )
  (policy_net): Sequential(
    (0): Linear(in_features=260, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
  )
  (action_net): Linear(in_features=64, out_features=24, bias=True)
)

In [ ]:
torch.save(agent.state_dict(), 'agent')

In [47]:
import visualization
import planning_on_voronoi

In [43]:
from gym import spaces
import warnings
import copy

In [44]:
'''
Utility functions from stable-baseline3.
See:
https://github.com/DLR-RM/stable-baselines3/blob/1cba1bbd2f129f3e3140d6a1e478dd4b3979a2bf/stable_baselines3/common/preprocessing.py
https://github.com/DLR-RM/stable-baselines3/blob/1cba1bbd2f129f3e3140d6a1e478dd4b3979a2bf/stable_baselines3/common/utils.py
https://github.com/DLR-RM/stable-baselines3/blob/1cba1bbd2f129f3e3140d6a1e478dd4b3979a2bf/stable_baselines3/common/policies.py
'''
def is_image_space_channels_first(observation_space: spaces.Box) -> bool:
    smallest_dimension = np.argmin(observation_space.shape).item()
    if smallest_dimension == 1:
        warnings.warn("Treating image space as channels-last, while second dimension was smallest of the three.")
    return smallest_dimension == 0


def is_image_space(
    observation_space: spaces.Space,
    check_channels: bool = False,
    normalized_image: bool = False,
) -> bool:
    check_dtype = check_bounds = not normalized_image
    if isinstance(observation_space, spaces.Box) and len(observation_space.shape) == 3:
        # Check the type
        if check_dtype and observation_space.dtype != np.uint8:
            return False

        # Check the value range
        incorrect_bounds = np.any(observation_space.low != 0) or np.any(observation_space.high != 255)
        if check_bounds and incorrect_bounds:
            return False

        # Skip channels check
        if not check_channels:
            return True
        # Check the number of channels
        if is_image_space_channels_first(observation_space):
            n_channels = observation_space.shape[0]
        else:
            n_channels = observation_space.shape[-1]
        # GrayScale, RGB, RGBD
        return n_channels in [1, 3, 4]
    return False

def maybe_transpose(observation: np.ndarray, observation_space: spaces.Space) -> np.ndarray:
    # Avoid circular import
    if is_image_space(observation_space):
        if not (observation.shape == observation_space.shape or observation.shape[1:] == observation_space.shape):
            # Try to re-order the channels
            transpose_obs = transpose_image(observation)
            if transpose_obs.shape == observation_space.shape or transpose_obs.shape[1:] == observation_space.shape:
                observation = transpose_obs
    return observation

def transpose_image(image: np.ndarray) -> np.ndarray:
    if len(image.shape) == 3:
        return np.transpose(image, (2, 0, 1))
    return np.transpose(image, (0, 3, 1, 2))
    
def obs_as_tensor(obs, device):
    if isinstance(obs, np.ndarray):
        return torch.as_tensor(obs, device=device).float()
    elif isinstance(obs, dict):
        return {key: torch.as_tensor(_obs, device=device).float() for (key, _obs) in obs.items()}
    else:
        raise Exception(f"Unrecognized type of observation {type(obs)}")

def obs_to_tensor(observation, observation_space, device):
    # need to copy the dict as the dict in VecFrameStack will become a torch tensor
    observation = copy.deepcopy(observation)
    for key, obs in observation.items():
        obs_space = observation_space.spaces[key]
        if is_image_space(obs_space):
            obs_ = maybe_transpose(obs, obs_space)
        else:
            obs_ = np.array(obs)
        # Add batch dimension if needed
        observation[key] = obs_.reshape((-1, *observation_space[key].shape))  # type: ignore[misc]

    obs_tensor = obs_as_tensor(observation, device)
    return obs_tensor

In [ ]:
'''
Test trained student policy.
'''
obs, _ = env.reset()
radar_config = env.radar_locs
print("Initial state: ", obs['state'])
trajectory = []
trajectory.append(obs['state'])

with torch.no_grad():
    for i in range(1500):
        action, _ = agent.forward(obs_to_tensor(obs, env.observation_space, configs.cfg.alg.device))
        action = action.cpu().numpy()
        # print(action[0])
        action_clipped = np.clip(action[0], env.action_space.low, env.action_space.high)
        obs, reward, done, _, _ = env.step(action[0])
        # vec_env.render()
        # print("Action: ", action)
        trajectory.append(env.state['state'])
        if done:
            # print("Action: ", action)
            print("State: ", env.state['state'])
            break
radar_locs, voronoi_diagram, path = planning_on_voronoi.get_baseline_path_with_vertices(radar_config, size_of_map)
visualization.visualiza_traj(trajectory, radar_locs, voronoi_diagram, path, save=True)